In [2]:
import pandas as pd
parquet_path = '/data/jupiter/datasets/all_jupiter_data_stratified/df_cache.parquet'
df_orig = pd.read_parquet(parquet_path)

In [4]:
useful_columns = ['pred_dust_percent', 'speed_discrete',  'dust>50', 'dust>35', 'dust>25', 'dust>15', 'dust>5',
                        'collected_on_localtime', 'camera_location', 'camera_location_grouped',
                        'latitude', 'longitude', 'robot_name', 'operation_time', 'state',
                        'id', 'collected_on_localtime', 'speed_kph']
df = df_orig[useful_columns]

In [5]:
import numpy as np
def percentage(c):
    return np.mean(c) * 100
agg_dict = {
    'dust>5': percentage,
    'dust>15': percentage,
    'dust>25': percentage,
    'dust>35': percentage,
    'dust>50': percentage,
    'id': 'size',
}
pd.set_option('display.max_rows', None)

In [19]:
# Already in the cached df, just for reference you can look at it
def get_region(state):
    match state:
        case 'Illinois' | 'Indiana' | 'Iowa' | 'Kansas' | 'Michigan' | 'Minnesota' | 'Missouri' | 'Nebraska' | 'North Dakota' | 'Ohio' | 'South Dakota' | 'Wisconsin':
            return 'midwest'
        case 'Alabama' | 'Arkansas' | 'Florida' | 'Georgia' | 'Kentucky' | 'Louisiana' | 'Oklahoma' | 'Texas':
            return 'southeast'
        case 'California' | 'Colorado' | 'Nevada' | 'New Mexico' | 'Oregon' | 'Utah' | 'Wyoming':
            return 'west'
        case 'unknown' | None:
            return 'unknown'
    print(state)
    assert False # 

In [22]:
df.groupby(['region']).agg(agg_dict).rename({'id': 'count'}, axis=1)

,dust>5,dust>15,dust>25,dust>35,dust>50,count
region,,,,,,
midwest,8.452354,5.275197,3.618216,1.164483,0.663230,91371
southeast,10.677664,6.167800,4.097470,1.954692,1.010087,71776
unknown,9.493236,6.337511,4.544055,2.163046,1.132460,42209
west,6.096779,4.115593,3.149943,2.116591,1.496579,28064


In [23]:
df.groupby(['operation_time', 'camera_location_grouped', 'speed_discrete']).agg(agg_dict).rename({'id': 'count'}, axis=1)

dust>5    dust>15  \
operation_time camera_location_grouped speed_discrete                         
daytime        halo_front              A:0-2            3.355215   0.802334   
                                       B:2-10           1.229050   0.670391   
                                       C:10-25          1.154250   0.209864   
                                       D:25+            3.174603   0.793651   
               halo_implement          A:0-2            5.606407   4.233410   
                                       B:2-10          14.285714   9.461967   
                                       C:10-25         38.111888  27.447552   
                                       D:25+           12.686567   4.850746   
               halo_rear               A:0-2            1.800554   0.484765   
                                       B:2-10           4.468085   1.808511   
                                       C:10-25         19.300000   8.900000   
                                       D:25+           11.190053   5.328597   
               halo_side               A:0-2            3.737024   2.422145   
                                       B:2-10           4.968619   2.928870   
                                       C:10-25          6.947891   3.771712   
                                       D:25+            4.102097   2.370100   
               rev1_front              A:0-2            0.498836   0.149651   
                                       B:2-10           0.209950   0.036513   
                                       C:10-25          0.343682   0.183297   
                                       D:25+            1.559322   0.610169   
               rev1_rear               A:0-2            1.106420   0.291163   
                                       B:2-10          10.486779   2.659255   
                                       C:10-25         16.595666   4.908316   
                                       D:25+           11.720430   4.516129   
               rev1_side_front         A:0-2            0.732845   0.295636   
                                       B:2-10           0.483996   0.136980   
                                       C:10-25          0.564516   0.144009   
                                       D:25+            0.304054   0.067568   
               rev1_side_rear          A:0-2            0.625322   0.132421   
                                       B:2-10           2.399697   0.817562   
                                       C:10-25          4.863867   2.141209   
                                       D:25+            3.598281   0.966702   
nightime       halo_front              A:0-2            1.081081   0.000000   
                                       B:2-10           6.329114   2.531646   
                                       C:10-25          2.797203   1.398601   
                                       D:25+           55.555556   0.000000   
               halo_implement          A:0-2            3.174603   3.174603   
                                       B:2-10          15.662651   8.433735   
                                       C:10-25         22.807018  14.912281   
                                       D:25+            0.000000   0.000000   
               halo_rear               A:0-2            0.000000   0.000000   
                                       B:2-10          10.638298   5.319149   
                                       C:10-25         18.439716  11.347518   
                                       D:25+            0.000000   0.000000   
               halo_side               A:0-2            0.712589   0.475059   
                                       B:2-10           5.147059   2.573529   
                                       C:10-25          8.056872   5.450237   
                                       D:25+            0.000000   0.000000   
               rev1_front              A:0-2           25.861141  17.841765   
                             

In [24]:
df[df['speed_discrete'] == 'C:10-25'].groupby(['region', 'operation_time']).agg(agg_dict).rename({'id': 'count'}, axis=1)

dust>5    dust>15    dust>25    dust>35  \
region    operation_time                                               
midwest   daytime          3.852255   1.858146   1.229322   0.781781   
          nightime        26.462510  18.140621  12.839879   3.735238   
southeast daytime          5.490599   1.832833   0.955917   0.458208   
          nightime        32.045586  20.977400  14.390574   7.552637   
unknown   daytime          4.760497   1.665681   0.857481   0.473093   
          nightime        50.674105  38.958624  28.730823  14.458391   
west      daytime          3.980022   2.075855   1.279850   0.952084   
          nightime        48.333333  37.777778  30.555556  21.296296   

                            dust>50     id  
region    operation_time                    
midwest   daytime          0.390891  17652  
          nightime         1.908816   7282  
southeast daytime          0.189603  12658  
          nightime         3.071277   5177  
unknown   daytime          0.246403  10146  
          nightime         7.717341   2151  
west      daytime          0.639925   6407  
          nightime        13.333333    540